## Validating your model

### Simple holdout validation
This is the simplest evaluation protocol, and it suffers from one flaw: if little data is available, then your validation and test sets may contain too few samples to be statistically representative of the data at hand. 

In [3]:
from tensorflow import keras
from tensorflow.keras.datasets import imdb
from tensorflow.keras import layers
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(
    num_words=10000)

model = keras.Sequential([
    layers.Dense(16, activation="relu"),
    layers.Dense(16, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])

model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])

import numpy as np 
def vectorize_sequences(sequences, dimension=10000): 
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        for j in sequence:
            results[i, j] = 1.
    return results
x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)
y_train = np.asarray(train_labels).astype("float32")
y_test = np.asarray(test_labels).astype("float32")

In [4]:
x_val = x_train[:10000]
partial_x_train = x_train[10000:]
y_val = y_train[:10000]
partial_y_train = y_train[10000:]

history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=20,
                    batch_size=512,
                    validation_data=(x_val, y_val))

Epoch 1/20
30/30 [==============================] - 1s 22ms/step - loss: 0.5282 - accuracy: 0.7883 - val_loss: 0.4460 - val_accuracy: 0.8165
Epoch 2/20
30/30 [==============================] - 0s 11ms/step - loss: 0.3385 - accuracy: 0.8897 - val_loss: 0.3205 - val_accuracy: 0.8845
Epoch 3/20
30/30 [==============================] - 0s 11ms/step - loss: 0.2536 - accuracy: 0.9157 - val_loss: 0.2903 - val_accuracy: 0.8852
Epoch 4/20
30/30 [==============================] - 0s 10ms/step - loss: 0.2071 - accuracy: 0.9297 - val_loss: 0.2805 - val_accuracy: 0.8869
Epoch 5/20
30/30 [==============================] - 0s 11ms/step - loss: 0.1719 - accuracy: 0.9436 - val_loss: 0.2731 - val_accuracy: 0.8887
Epoch 6/20
30/30 [==============================] - 0s 11ms/step - loss: 0.1465 - accuracy: 0.9525 - val_loss: 0.2820 - val_accuracy: 0.8861
Epoch 7/20
30/30 [==============================] - 0s 11ms/step - loss: 0.1290 - accuracy: 0.9597 - val_loss: 0.2889 - val_accuracy: 0.8852
Epoch 8/20
30

### K-fold validation
With this approach, you split your data into K partitions of equal size. For each partition i, train a model on the remaining K - 1 partitions, and evaluate it on partition i

In [6]:
def get_model():
    model = keras.Sequential([
        layers.Dense(16, activation="relu"),
        layers.Dense(16, activation="relu"),
        layers.Dense(1, activation="sigmoid")
    ])

    model.compile(optimizer="rmsprop",
                  loss="binary_crossentropy",
                  metrics=["accuracy"])
    return model

In [20]:
import tensorflow as tf
import numpy as np
y_train_two = np.reshape(y_train, (y_train.shape[0], 1))
print(f"{x_train.shape}, {y_train_two.shape}")
data = np.concatenate((x_train, y_train_two), axis=1)
print(data)

(25000, 10000), (25000, 1)
[[0. 1. 1. ... 0. 0. 1.]
 [0. 1. 1. ... 0. 0. 0.]
 [0. 1. 1. ... 0. 0. 0.]
 ...
 [0. 1. 1. ... 0. 0. 0.]
 [0. 1. 1. ... 0. 0. 1.]
 [0. 1. 1. ... 0. 0. 0.]]


In [28]:

k = 3 
num_validation_samples = len(data) // k
np.random.shuffle(data)
all_acc_histories = [] 
validation_scores = [] 
for fold in range(k):
    validation_data = data[num_validation_samples * fold:
                           num_validation_samples * (fold + 1)]
    training_data = np.concatenate((
        data[:num_validation_samples * fold],
        data[num_validation_samples * (fold + 1):]))
    model = get_model()
    history = model.fit(training_data[:, :-1],
                        training_data[:, -1],
                    epochs=20,
                    batch_size=512,
                    validation_data=(validation_data[:, :-1],validation_data[:, -1]))
    acc_history = history.history["val_accuracy"]
    all_acc_histories.append(acc_history)
    print(acc_history)

print(all_acc_histories)

0
8333
Epoch 1/20
33/33 [==============================] - 1s 26ms/step - loss: 0.5550 - accuracy: 0.7788 - val_loss: 0.4302 - val_accuracy: 0.8625
Epoch 2/20
33/33 [==============================] - 0s 11ms/step - loss: 0.3525 - accuracy: 0.8880 - val_loss: 0.3272 - val_accuracy: 0.8798
Epoch 3/20
33/33 [==============================] - 0s 11ms/step - loss: 0.2630 - accuracy: 0.9098 - val_loss: 0.2876 - val_accuracy: 0.8908
Epoch 4/20
33/33 [==============================] - 0s 10ms/step - loss: 0.2137 - accuracy: 0.9267 - val_loss: 0.2746 - val_accuracy: 0.8937
Epoch 5/20
33/33 [==============================] - 0s 11ms/step - loss: 0.1808 - accuracy: 0.9384 - val_loss: 0.2771 - val_accuracy: 0.8948
Epoch 6/20
33/33 [==============================] - 0s 11ms/step - loss: 0.1545 - accuracy: 0.9491 - val_loss: 0.2816 - val_accuracy: 0.8896
Epoch 7/20
33/33 [==============================] - 0s 11ms/step - loss: 0.1345 - accuracy: 0.9564 - val_loss: 0.2888 - val_accuracy: 0.8909
Epoch 

33/33 [==============================] - 0s 10ms/step - loss: 0.0431 - accuracy: 0.9899 - val_loss: 0.4523 - val_accuracy: 0.8721
Epoch 14/20
33/33 [==============================] - 0s 10ms/step - loss: 0.0343 - accuracy: 0.9930 - val_loss: 0.4705 - val_accuracy: 0.8723
Epoch 15/20
33/33 [==============================] - 0s 10ms/step - loss: 0.0304 - accuracy: 0.9939 - val_loss: 0.5070 - val_accuracy: 0.8690
Epoch 16/20
33/33 [==============================] - 0s 10ms/step - loss: 0.0275 - accuracy: 0.9948 - val_loss: 0.5164 - val_accuracy: 0.8741
Epoch 17/20
33/33 [==============================] - 0s 10ms/step - loss: 0.0195 - accuracy: 0.9975 - val_loss: 0.5489 - val_accuracy: 0.8706
Epoch 18/20
33/33 [==============================] - 0s 10ms/step - loss: 0.0187 - accuracy: 0.9969 - val_loss: 0.5706 - val_accuracy: 0.8716
Epoch 19/20
33/33 [==============================] - 0s 10ms/step - loss: 0.0147 - accuracy: 0.9978 - val_loss: 0.6318 - val_accuracy: 0.8632
Epoch 20/20
33/33 

In [29]:
print(np.mean(all_acc_histories))

0.8762170523405075


### Iterated K-fold validation with shuffling
This one is for situations in which you have relatively little data available and you need to evaluate your model as precisely as possible. I’ve found it to be extremely helpful in Kaggle competitions. It consists of applying K-fold validation multiple times, shuffling the data every time before splitting it K ways. The final score is the average of the scores obtained at each run of K-fold validation. Note that you end up training and evaluating P * K models (where P is the number of iterations you use), which can be very expensive.